In [111]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('csv'):
            print(os.path.join(dirname, filename))

             
print()

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '__output__' not in filename:
            if filename.endswith('json'):
                print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

/kaggle/input/covertype-target/target.csv
/kaggle/input/forest-cover-type-kernels-only/sample_submission.csv
/kaggle/input/forest-cover-type-kernels-only/test.csv
/kaggle/input/forest-cover-type-kernels-only/train.csv

/kaggle/input/featuretools-feature-selection-multiply-2/features.json
/kaggle/input/featuretools-feature-selection-diff-2/features.json
/kaggle/input/featuretools-feature-selection-add-2/features.json
/kaggle/input/featuretools-feature-selection-abs-diff/features.json
/kaggle/input/featuretools-feature-selection-div-2/features.json
/kaggle/input/featuretools-feature-selection/features.json
/kaggle/input/featuretools-feature-selection-squred-root/features.json
/kaggle/input/featuretools-feature-selection-mean-2/features.json


In [112]:
%%time
train_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/train.csv')
test_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/test.csv')
submission_df = pd.read_csv('/kaggle/input/forest-cover-type-kernels-only/sample_submission.csv')

CPU times: user 1.76 s, sys: 340 ms, total: 2.1 s
Wall time: 1.94 s


In [113]:
train_df.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [114]:
test_df['Cover_Type'] = np.nan
data_df = pd.concat([train_df, test_df]).drop(columns=['Cover_Type'])

In [115]:
print(train_df.shape)
print(test_df.shape)
print(data_df.shape)

(15120, 56)
(565892, 56)
(581012, 55)


## Read Target Dataset

In [116]:
target_all = pd.read_csv('/kaggle/input/covertype-target/target.csv',
                        index_col='Id')

y_test = target_all.loc[test_df['Id']]

## Convert One-Hot encoding to label

In [117]:

cat_soil = [col for col in data_df.columns if 'Soil' in col]
data_df['Soil_Type'] = data_df[cat_soil].values.argmax(axis=1)+1
data_df = data_df.drop(columns=cat_soil)


cat_wilderness = [col for col in data_df.columns if 'Wilderness' in col]
data_df['Wilderness_Area'] = data_df[cat_wilderness].values.argmax(axis=1)+1
data_df = data_df.drop(columns=cat_wilderness)

In [118]:
data_df.shape

(581012, 13)

In [119]:
data_df.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Soil_Type',
       'Wilderness_Area'],
      dtype='object')

In [120]:
train_df.columns

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_

In [121]:
import featuretools as ft
import featuretools.variable_types as vtypes
from multiprocessing import cpu_count

In [122]:
from IPython.display import display

for i in range(0, len(ft.list_primitives()), 10):
#     print(i)
    
    display(ft.list_primitives()[0+i:i+10])

,name,type,description
0,min,aggregation,"Calculates the smallest value, ignoring `NaN` values."
1,skew,aggregation,Computes the extent to which a distribution differs from a normal distribution.
2,avg_time_between,aggregation,Computes the average number of seconds between consecutive events.
3,median,aggregation,Determines the middlemost number in a list of values.
4,mode,aggregation,Determines the most commonly repeated value.
5,n_most_common,aggregation,Determines the `n` most common elements.
6,time_since_last,aggregation,Calculates the time elapsed since the last datetime (default in seconds).
7,mean,aggregation,Computes the average for a list of values.
8,trend,aggregation,Calculates the trend of a variable over time.
9,percent_true,aggregation,Determines the percent of `True` values.


,name,type,description
10,all,aggregation,Calculates if all values are 'True' in a list.
11,sum,aggregation,"Calculates the total addition, ignoring `NaN`."
12,num_true,aggregation,Counts the number of `True` values.
13,any,aggregation,Determines if any value is 'True' in a list.
14,count,aggregation,"Determines the total number of values, excluding `NaN`."
15,num_unique,aggregation,"Determines the number of distinct values, ignoring `NaN` values."
16,last,aggregation,Determines the last value in a list.
17,max,aggregation,"Calculates the highest value, ignoring `NaN` values."
18,std,aggregation,"Computes the dispersion relative to the mean value, ignoring `NaN`."
19,time_since_first,aggregation,Calculates the time elapsed since the first datetime (in seconds).


,name,type,description
20,greater_than,transform,Determines if values in one list are greater than another list.
21,modulo_by_feature,transform,Return the modulo of a scalar by each element in the list.
22,subtract_numeric_scalar,transform,Subtract a scalar from each element in the list.
23,divide_by_feature,transform,Divide a scalar by each value in the list.
24,modulo_numeric,transform,Element-wise modulo of two lists.
25,week,transform,Determines the week of the year from a datetime.
26,cum_max,transform,Calculates the cumulative maximum.
27,percentile,transform,Determines the percentile rank for each value in a list.
28,haversine,transform,Calculates the approximate haversine distance between two LatLong
29,second,transform,Determines the seconds value of a datetime.


,name,type,description
30,cum_mean,transform,Calculates the cumulative mean.
31,equal_scalar,transform,Determines if values in a list are equal to a given scalar.
32,negate,transform,Negates a numeric value.
33,subtract_numeric,transform,Element-wise subtraction of two lists.
34,time_since,transform,Calculates time from a value to a specified cutoff datetime.
35,day,transform,Determines the day of the month from a datetime.
36,or,transform,Element-wise logical OR of two lists.
37,cum_sum,transform,Calculates the cumulative sum.
38,less_than_equal_to_scalar,transform,Determines if values are less than or equal to a given scalar.
39,year,transform,Determines the year value of a datetime.


,name,type,description
40,add_numeric,transform,Element-wise addition of two lists.
41,isin,transform,Determines whether a value is present in a provided list.
42,num_characters,transform,Calculates the number of characters in a string.
43,less_than_scalar,transform,Determines if values are less than a given scalar.
44,time_since_previous,transform,Compute the time since the previous entry in a list.
45,greater_than_equal_to_scalar,transform,Determines if values are greater than or equal to a given scalar.
46,multiply_numeric_scalar,transform,Multiply each element in the list by a scalar.
47,not_equal,transform,Determines if values in one list are not equal to another list.
48,is_null,transform,Determines if a value is null.
49,greater_than_scalar,transform,Determines if values are greater than a given scalar.


,name,type,description
50,and,transform,Element-wise logical AND of two lists.
51,scalar_subtract_numeric_feature,transform,Subtract each value in the list from a given scalar.
52,weekday,transform,Determines the day of the week from a datetime.
53,modulo_numeric_scalar,transform,Return the modulo of each element in the list by a scalar.
54,divide_numeric_scalar,transform,Divide each element in the list by a scalar.
55,latitude,transform,Returns the first tuple value in a list of LatLong tuples.
56,minute,transform,Determines the minutes value of a datetime.
57,cum_min,transform,Calculates the cumulative minimum.
58,not,transform,Negates a boolean value.
59,hour,transform,Determines the hour value of a datetime.


,name,type,description
60,cum_count,transform,Calculates the cumulative count.
61,equal,transform,Determines if values in one list are equal to another list.
62,is_weekend,transform,Determines if a date falls on a weekend.
63,add_numeric_scalar,transform,Add a scalar to each value in the list.
64,diff,transform,Compute the difference between the value in a list and the
65,num_words,transform,Determines the number of words in a string by counting the spaces.
66,less_than,transform,Determines if values in one list are less than another list.
67,less_than_equal_to,transform,Determines if values in one list are less than or equal to another list.
68,divide_numeric,transform,Element-wise division of two lists.
69,month,transform,Determines the month value of a datetime.


,name,type,description
70,not_equal_scalar,transform,Determines if values in a list are not equal to a given scalar.
71,longitude,transform,Returns the second tuple value in a list of LatLong tuples.
72,absolute,transform,Computes the absolute value of a number.
73,greater_than_equal_to,transform,Determines if values in one list are greater than or equal to another list.
74,multiply_numeric,transform,Element-wise multiplication of two lists.


# Feature Engineering

### Define Transform Primitives

In [123]:
from featuretools.primitives import make_trans_primitive
from featuretools.variable_types import Numeric

def squared_root(col1, col2):
    return (col1**2 + col2**2)**0.5

Squared_Root = make_trans_primitive(function=squared_root,
                                   input_types=[Numeric, Numeric],
                                   return_type=Numeric,
                                    commutative=True, # True: combination, False: permutaion
                                   )


def abs_diff(col1, col2):
    return abs(col1 - col2)

Abs_Diff = make_trans_primitive(function=abs_diff,
                               input_types=[Numeric, Numeric],
                               return_type=Numeric,
                               commutative=True, # True: combination, False: permutaion
                               )


def mean_2_cols(col1, col2):
    return (col1.add(col2)) / 2


Mean_2_Cols = make_trans_primitive(function=mean_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def mean_3_cols(col1, col2, col3):
    return (col1.add(col2.add(col3))) / 3

Mean_3_Cols = make_trans_primitive(function=mean_3_cols,
                                  input_types=[Numeric, Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def div_2_cols(col1, col2):
    eps = 10e-2
    return (col1+eps) / (col2+eps)


Div_2_Cols = make_trans_primitive(function=div_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )

### Read Feature Json Files

In [124]:
from collections import OrderedDict

filename_dict = OrderedDict()

filename_dict['add_2'] ='/kaggle/input/featuretools-feature-selection-add-2/features.json'
filename_dict['dif_2'] = '/kaggle/input/featuretools-feature-selection-diff-2/features.json'
filename_dict['abs_dif_2'] ='/kaggle/input/featuretools-feature-selection-abs-diff/features.json'
# filename_dict['mul_2'] = '/kaggle/input/featuretools-feature-selection-multiply-2/features.json'
filename_dict['div_2'] ='/kaggle/input/featuretools-feature-selection-div-2/features.json'
filename_dict['mean_2'] ='/kaggle/input/featuretools-feature-selection-mean-2/features.json'
filename_dict['mean_3'] ='/kaggle/input/featuretools-feature-selection/features.json'
filename_dict['sqrt_2'] ='/kaggle/input/featuretools-feature-selection-squred-root/features.json'

In [125]:
n_features = 0

features_list = []
features_dict = OrderedDict()
for name, filename in filename_dict.items():
    features_dict[name] = ft.load_features(filename)
    features_list = features_list + features_dict[name]
    
    print('{:10s}: num of features= {}'.format(name,
                                       len(features_dict[name])))
    
    n_features += len(features_dict[name])
    
print()
print('Total num of features=       {}'.format(n_features))

add_2     : num of features= 13
dif_2     : num of features= 12
abs_dif_2 : num of features= 13
div_2     : num of features= 10
mean_2    : num of features= 12
mean_3    : num of features= 10
sqrt_2    : num of features= 11

Total num of features=       81


In [126]:
variable_type = {col: vtypes.Categorical for col in data_df.columns if 'Soil' in col or 'Wilderness' in col}

# variable_type['Cover_Type'] = vtypes.Categorical

print(variable_type)

{'Soil_Type': <class 'featuretools.variable_types.variable.Categorical'>, 'Wilderness_Area': <class 'featuretools.variable_types.variable.Categorical'>}


In [127]:
es = ft.EntitySet('Dataset')

es.entity_from_dataframe(entity_id='forest_table',
                         index='Id',
                        dataframe=data_df,
                         variable_types=variable_type
                        )

Entityset: Dataset
  Entities:
    forest_table [Rows: 581012, Columns: 13]
  Relationships:
    No relationships

In [128]:
es['forest_table'].variables

[<Variable: Id (dtype = index)>,
 <Variable: Elevation (dtype = numeric)>,
 <Variable: Aspect (dtype = numeric)>,
 <Variable: Slope (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Hydrology (dtype = numeric)>,
 <Variable: Vertical_Distance_To_Hydrology (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Roadways (dtype = numeric)>,
 <Variable: Hillshade_9am (dtype = numeric)>,
 <Variable: Hillshade_Noon (dtype = numeric)>,
 <Variable: Hillshade_3pm (dtype = numeric)>,
 <Variable: Horizontal_Distance_To_Fire_Points (dtype = numeric)>,
 <Variable: Soil_Type (dtype = categorical)>,
 <Variable: Wilderness_Area (dtype = categorical)>]

In [129]:
es['forest_table'].df

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area
1,1,2596,51,3,258,0,510,221,232,148,6279,29,1
2,2,2590,56,2,212,-6,390,220,235,151,6225,29,1
3,3,2804,139,9,268,65,3180,234,238,135,6121,12,1
4,4,2785,155,18,242,118,3090,238,238,122,6211,30,1
5,5,2595,45,2,153,-1,391,220,234,150,6172,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581008,581008,2396,153,20,85,17,108,240,237,118,837,2,3
581009,581009,2391,152,19,67,12,95,240,237,119,845,2,3
581010,581010,2386,159,17,60,7,90,236,241,130,854,2,3
581011,581011,2384,170,15,60,5,90,230,245,143,864,2,3


In [130]:
%%time
_, features_org = ft.dfs(entityset=es,
                        target_entity='forest_table')

features_list_combined = features_org + features_list

print('original features:           {}'.format(len(features_org)))
print('selected features from fils: {}'.format(len(features_list)))
print('combined features:           {}'.format(len(features_list_combined)))

print()
print('unique num of features:      {}'.format(len(set(features_list_combined))))
print()

original features:           12
selected features from fils: 81
combined features:           93

unique num of features:      93

CPU times: user 4.27 s, sys: 400 ms, total: 4.67 s
Wall time: 4.32 s


In [131]:
%%time
features = features_list_combined
feature_matrix = ft.calculate_feature_matrix(features,
                                            entityset=es,
#                                             n_jobs=cpu_count()
                                            )

CPU times: user 6.29 s, sys: 2.65 s, total: 8.94 s
Wall time: 6.67 s


In [132]:
# %%time
# feature_matrix, features = ft.dfs(entityset=es,
#                                  target_entity='forest_table',
#                                   trans_primitives=[
# #                                                     Squared_Root,
# #                                                     Abs_Diff,
#                                                     Mean_3_Cols,
# #                                                       Div_2_Cols,
# #                                                     'add_numeric', 
# #                                                   'subtract_numeric',
# #                                                     'multiply_numeric',
# #                                                     'divide_numeric',  # divide by 0.
# #                                                     'subtract_numeric',
#                                                    ],
#                                  max_depth=1,
#                                  n_jobs=cpu_count())

In [133]:
feature_matrix = feature_matrix.reset_index()

In [134]:
feature_matrix.columns[:5]

Index(['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology'],
      dtype='object')

In [135]:
feature_matrix.shape

(581012, 94)

In [136]:
data_df.shape

(581012, 13)

In [137]:
features

[<Feature: Elevation>,
 <Feature: Aspect>,
 <Feature: Slope>,
 <Feature: Horizontal_Distance_To_Hydrology>,
 <Feature: Vertical_Distance_To_Hydrology>,
 <Feature: Horizontal_Distance_To_Roadways>,
 <Feature: Hillshade_9am>,
 <Feature: Hillshade_Noon>,
 <Feature: Hillshade_3pm>,
 <Feature: Horizontal_Distance_To_Fire_Points>,
 <Feature: Soil_Type>,
 <Feature: Wilderness_Area>,
 <Feature: Horizontal_Distance_To_Hydrology + Horizontal_Distance_To_Roadways>,
 <Feature: Aspect + Horizontal_Distance_To_Hydrology>,
 <Feature: Elevation + Horizontal_Distance_To_Fire_Points>,
 <Feature: Aspect + Vertical_Distance_To_Hydrology>,
 <Feature: Aspect + Horizontal_Distance_To_Roadways>,
 <Feature: Elevation + Horizontal_Distance_To_Hydrology>,
 <Feature: Aspect + Elevation>,
 <Feature: Hillshade_3pm + Horizontal_Distance_To_Hydrology>,
 <Feature: Elevation + Vertical_Distance_To_Hydrology>,
 <Feature: Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Hydrology>,
 <Feature: Horizontal_Distan

# Prepare Train/Test Dataset

In [138]:
feature_matrix = feature_matrix.set_index('Id')

In [139]:
indice = train_df['Id']
# X_train = feature_matrix.loc[indice].drop(columns=['soil_type_table.Cover_Type'])
# y_train = feature_matrix.loc[indice, 'soil_type_table.Cover_Type'].astype('category')
# y_train = train_df['Cover_Type']
X_train = feature_matrix.loc[indice]
y_train = train_df['Cover_Type'].astype('category')

indice = test_df['Id']
# X_test = feature_matrix.loc[indice].drop(columns=['soil_type_table.Cover_Type'])
X_test = feature_matrix.loc[indice]

### Recycle `feature_matrix`

In [140]:
import gc

del feature_matrix
# del features

gc.collect()

536

In [141]:
from IPython.display import display

display(X_train.tail())
display(train_df.tail())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"SQUARED_ROOT(Elevation, Hillshade_3pm)","SQUARED_ROOT(Aspect, Elevation)","SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Vertical_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Slope)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)"
Id,,,,,,,,,,,,,,,,,,,,,
15116,2607,243,23,258,7,660,170,251,214,1282,...,2615.768530,2618.300594,1307.703330,2607.009398,2619.735292,1441.916780,708.635308,2607.101456,2905.163162,2689.246921
15117,2603,121,19,633,195,618,249,221,91,1325,...,2604.590179,2605.810814,1468.439308,2610.293853,2678.861325,1462.035909,884.654170,2603.069342,2920.827622,2675.356612
15118,2492,134,25,365,117,335,250,220,83,1187,...,2493.381840,2495.600128,1241.851038,2494.745077,2518.588692,1233.366936,495.429107,2492.125398,2760.259589,2514.416234
15119,2487,167,28,218,101,242,229,237,119,932,...,2489.845377,2492.600650,957.156205,2489.050020,2496.536200,962.906018,325.711529,2487.157615,2655.897777,2498.746286
15120,2475,197,34,319,78,270,189,244,164,914,...,2480.427584,2482.827823,968.068696,2476.228786,2495.473101,953.045644,417.924634,2475.233524,2638.374689,2489.683715


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
15115,15116,2607,243,23,258,7,660,170,251,214,...,0,0,0,0,0,0,0,0,0,3
15116,15117,2603,121,19,633,195,618,249,221,91,...,0,0,0,0,0,0,0,0,0,3
15117,15118,2492,134,25,365,117,335,250,220,83,...,0,0,0,0,0,0,0,0,0,3
15118,15119,2487,167,28,218,101,242,229,237,119,...,0,0,0,0,0,0,0,0,0,3
15119,15120,2475,197,34,319,78,270,189,244,164,...,0,0,0,0,0,0,0,0,0,3


In [142]:
display(X_test.tail())
display(test_df.tail())

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"SQUARED_ROOT(Elevation, Hillshade_3pm)","SQUARED_ROOT(Aspect, Elevation)","SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Vertical_Distance_To_Hydrology)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Hydrology)","SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Slope)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)"
Id,,,,,,,,,,,,,,,,,,,,,
581008,2396,153,20,85,17,108,240,237,118,837,...,2398.903916,2400.880047,841.304939,2396.060308,2397.507247,843.938979,137.437258,2396.083471,2537.988377,2398.432822
581009,2391,152,19,67,12,95,240,237,119,845,...,2393.959482,2395.826580,847.652051,2391.030113,2391.938544,850.323468,116.249731,2391.075490,2535.923106,2392.886541
581010,2386,159,17,60,7,90,236,241,130,854,...,2389.538868,2391.291910,856.105134,2386.010268,2386.754281,858.729294,108.166538,2386.060561,2534.228088,2387.696798
581011,2384,170,15,60,5,90,230,245,143,864,...,2388.284949,2390.053556,866.080828,2384.005243,2384.754914,868.674853,108.166538,2384.047189,2535.735002,2385.698221
581012,2383,165,13,60,4,67,231,244,141,875,...,2387.167778,2388.705507,877.054730,2383.003357,2383.755231,877.561394,89.938868,2383.035459,2538.565343,2383.941694


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
565887,581008,2396,153,20,85,17,108,240,237,118,...,0,0,0,0,0,0,0,0,0,NaN
565888,581009,2391,152,19,67,12,95,240,237,119,...,0,0,0,0,0,0,0,0,0,NaN
565889,581010,2386,159,17,60,7,90,236,241,130,...,0,0,0,0,0,0,0,0,0,NaN
565890,581011,2384,170,15,60,5,90,230,245,143,...,0,0,0,0,0,0,0,0,0,NaN
565891,581012,2383,165,13,60,4,67,231,244,141,...,0,0,0,0,0,0,0,0,0,NaN


# Training

In [143]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score

from tqdm import tqdm_notebook

from collections import OrderedDict

def fit_predict(model, X_train=X_train, y_train=y_train, 
                        X_test=X_test, y_test=y_test):
    
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    score = accuracy_score(predictions, y_test)
    
    return score


def fit_pipelines(pipelines, 
                  X_train=X_train, y_train=y_train,
                 X_test=X_test, y_test=y_test):
    
    scores = {}
    for name, pipeline in tqdm_notebook(pipelines.items()):
        print('Start fitting using {}'.format(name))
        pipeline.fit(X_train, y_train)

        predictions = pipeline.predict(X_test)
        score = accuracy_score(predictions, y_test)
        scores[name] = score

        print('{:20s}: score={}'.format(name, score))

    return(scores)

In [144]:
%%time


seed=47
pipelines = {}

pipelines['Random Forest'] = Pipeline(steps=[('RF', 
                                            RandomForestClassifier(n_jobs=-1,
                                                                   n_estimators=300,
#                                                                    max_features=7,
#                                                                    max_leaf_nodes=220,
                                                                   random_state=seed
                                                                   )
                                                    ),])

pipelines['Extra Trees'] = Pipeline(
                                    steps=[('random forest extra', ExtraTreesClassifier(n_estimators=100, random_state=seed))]
                                    )

pipelines['KNN'] = Pipeline(steps=[
                                        ('KNN', KNeighborsClassifier(n_neighbors=2))
                                    ])

pipelines['LGBM'] = Pipeline(steps=[('LightGBM', 
                                    lgb.LGBMClassifier(n_estimators=300,
                                                             num_class=8,
                                                            num_leaves=32,
                                                            random_state=seed,)
                                   ),])



scores = fit_pipelines(pipelines)

Start fitting using Random Forest
Random Forest       : score=0.7590193888586515
Start fitting using Extra Trees
Extra Trees         : score=0.7860457472450574
Start fitting using KNN
KNN                 : score=0.6948764075123876
Start fitting using LGBM
LGBM                : score=0.7776077414064875

CPU times: user 10min 49s, sys: 5.72 s, total: 10min 55s
Wall time: 6min 37s


# Feature Selection

In [145]:
from sklearn.feature_selection import SelectFromModel


In [146]:
features_var = pd.DataFrame(index=X_train.columns)
name_classifiers = []

for name, p in tqdm_notebook(pipelines.items()):
    
    # cannot apply feature selection to KNN
    if 'KNN'.lower() in name.lower():
        continue

    name_classifiers.append(name)
    
#     step_name = p.steps[-1][0]
#     print(step_name)
    clf = p.steps[-1][1]
#     print(clf)

    model = SelectFromModel(clf, prefit=True)
    X_train_new = model.transform(X_train)
    X_train_inversed = model.inverse_transform(X_train_new)
    selected_features = pd.DataFrame(X_train_inversed,
                                    index=X_train.index,
                                    columns=X_train.columns)
    
    features_var[name] = selected_features.var()
    

In [147]:
features_var['mean'] = features_var[name_classifiers].mean(axis=1)
features_var['var'] = features_var[name_classifiers].var(axis=1)

features_var[:20]

,Random Forest,Extra Trees,LGBM,mean,var
Elevation,174455.068186,174455.068186,0.000000e+00,116303.378791,1.014486e+10
Aspect,0.000000,0.000000,1.211888e+04,4039.627889,4.895578e+07
Slope,0.000000,0.000000,7.146888e+01,23.822959,1.702600e+03
Horizontal_Distance_To_Hydrology,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
Vertical_Distance_To_Hydrology,0.000000,0.000000,3.750265e+03,1250.088288,4.688162e+06
Horizontal_Distance_To_Roadways,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
Hillshade_9am,933.992256,933.992256,9.339923e+02,933.992256,0.000000e+00
Hillshade_Noon,0.000000,0.000000,5.199296e+02,173.309878,9.010894e+04
Hillshade_3pm,0.000000,0.000000,2.106368e+03,702.122782,1.478929e+06
Horizontal_Distance_To_Fire_Points,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00


In [148]:
display(pd.DataFrame(features_var['mean'].sort_values(ascending=False)))
display(pd.DataFrame(features_var['var'].sort_values(ascending=False)))

,mean
"DIV_2_COLS(Elevation, Horizontal_Distance_To_Hydrology)",3.868997e+07
"DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)",1.356844e+07
"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",7.887222e+05
Horizontal_Distance_To_Hydrology + Horizontal_Distance_To_Roadways,6.377231e+05
Elevation + Horizontal_Distance_To_Fire_Points,5.972929e+05
...,...
"MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)",0.000000e+00
Aspect - Horizontal_Distance_To_Fire_Points,0.000000e+00
"DIV_2_COLS(Elevation, Horizontal_Distance_To_Roadways)",0.000000e+00
"MEAN_2_COLS(Elevation, Horizontal_Distance_To_Roadways)",0.000000e+00


,var
"DIV_2_COLS(Elevation, Horizontal_Distance_To_Hydrology)",1.122685e+15
"DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)",1.380770e+14
"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",1.866248e+12
Horizontal_Distance_To_Hydrology + Horizontal_Distance_To_Roadways,1.220072e+12
Elevation + Horizontal_Distance_To_Fire_Points,1.070276e+12
...,...
"MEAN_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)",0.000000e+00
"MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways, Vertical_Distance_To_Hydrology)",0.000000e+00
Elevation - Hillshade_3pm,0.000000e+00
Aspect - Elevation,0.000000e+00


### Show Top-10 mean and variation

In [149]:
# num_selected_features = 50
# num_selected_features = 30
# num_selected_features = 20  #Extra Trees         : score=0.8030701971400903
num_selected_features = 16 # Extra Trees         : score=0.8055035236405533
# num_selected_features = 13 # Extra Trees         : score=0.8048373187816756
# num_selected_features = 10 # Extra Trees         : score=0.8023156361991334


original_features = list(data_df.columns)
original_features.remove('Id')
original_features = set(original_features)

engineered_features = set(X_train) - original_features

top_mean = features_var['mean'].sort_values(ascending=False)[:num_selected_features]
top_var = features_var['var'].sort_values(ascending=False)[:num_selected_features]

top_mean_features = set(top_mean.index).intersection(engineered_features)
top_var_features = set(top_var.index).intersection(engineered_features)

print('original_features: {}'.format(len(original_features)))
print('top_mean_features: {}'.format(len(top_mean_features)))
print('top_var_features: {}'.format(len(top_var_features)))

print('top_mean_features + top_var_features: {}'.format(len(
                                                            top_mean_features.union(top_var_features)
                                                           
                                                           )))


selected_top_features = original_features.union(
                                                top_mean_features.union(top_var_features)
                                               )

print('Total Features: {}'.format(len(selected_top_features)))
print()
print(top_mean_features)
print()
print(top_var_features)
print()
print(top_mean_features.union(top_var_features))
print()

original_features: 12
top_mean_features: 16
top_var_features: 16
top_mean_features + top_var_features: 16
Total Features: 28

{'SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)', 'DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)', 'Horizontal_Distance_To_Fire_Points - Horizontal_Distance_To_Roadways', 'Elevation + Horizontal_Distance_To_Fire_Points', 'Elevation - Horizontal_Distance_To_Roadways', 'ABS_DIFF(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)', 'Horizontal_Distance_To_Hydrology - Horizontal_Distance_To_Roadways', 'SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)', 'Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Hydrology', 'DIV_2_COLS(Elevation, Horizontal_Distance_To_Hydrology)', 'SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)', 'Horizontal_Distance_To_Fire_Points - Horizontal_Distance_To_Hydrology', 'MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Poin

In [150]:
pd.options.display.max_colwidth = 100
display(pd.DataFrame(top_mean_features))
display(pd.DataFrame(top_var_features))

,0
0,"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)"
1,"DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)"
2,Horizontal_Distance_To_Fire_Points - Horizontal_Distance_To_Roadways
3,Elevation + Horizontal_Distance_To_Fire_Points
4,Elevation - Horizontal_Distance_To_Roadways
5,"ABS_DIFF(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)"
6,Horizontal_Distance_To_Hydrology - Horizontal_Distance_To_Roadways
7,"SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)"
8,Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Hydrology
9,"DIV_2_COLS(Elevation, Horizontal_Distance_To_Hydrology)"


,0
0,"SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)"
1,"DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)"
2,Horizontal_Distance_To_Fire_Points - Horizontal_Distance_To_Roadways
3,Elevation + Horizontal_Distance_To_Fire_Points
4,Elevation - Horizontal_Distance_To_Roadways
5,"ABS_DIFF(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)"
6,Horizontal_Distance_To_Hydrology - Horizontal_Distance_To_Roadways
7,"SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)"
8,Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Hydrology
9,"DIV_2_COLS(Elevation, Horizontal_Distance_To_Hydrology)"


# Re-Train using selected features

In [151]:
fit_pipelines(pipelines, 
                X_train=X_train[selected_top_features], y_train=y_train,
                 X_test=X_test[selected_top_features], y_test=y_test)

Start fitting using Random Forest
Random Forest       : score=0.7805676701561429
Start fitting using Extra Trees
Extra Trees         : score=0.8055035236405533
Start fitting using KNN
KNN                 : score=0.6656782566284732
Start fitting using LGBM
LGBM                : score=0.775278675082878



{'Random Forest': 0.7805676701561429,
 'Extra Trees': 0.8055035236405533,
 'KNN': 0.6656782566284732,
 'LGBM': 0.775278675082878}

# Save Features

In [152]:
feature_objs = []

all_selected_features = list(top_mean_features.union(top_var_features))
for f in features:
    if f.get_name() in all_selected_features:
        feature_objs.append(f)

In [153]:
len(all_selected_features)

16

In [154]:
ft.save_features(feature_objs,
                location='./features.json')

### Calculating Feature Matrix for New Data

In [155]:
saved_features = ft.load_features('./features.json')

#### calculate the feature matrix.

In [156]:
%%time
feature_matrix_restored = ft.calculate_feature_matrix(saved_features, es)

CPU times: user 4.79 s, sys: 348 ms, total: 5.14 s
Wall time: 4.5 s


In [157]:
feature_matrix_restored.shape

(581012, 16)

In [158]:
feature_matrix_restored.head()

,Horizontal_Distance_To_Hydrology + Horizontal_Distance_To_Roadways,Elevation + Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Fire_Points + Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Hydrology - Horizontal_Distance_To_Roadways,Horizontal_Distance_To_Fire_Points - Horizontal_Distance_To_Roadways,Elevation - Horizontal_Distance_To_Fire_Points,Elevation - Horizontal_Distance_To_Roadways,Horizontal_Distance_To_Fire_Points - Horizontal_Distance_To_Hydrology,"ABS_DIFF(Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","ABS_DIFF(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)","DIV_2_COLS(Elevation, Horizontal_Distance_To_Hydrology)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)"
Id,,,,,,,,,,,,,,,,
1,768,8875,6537,-252,5769,-3683,2086,6021,252,5769,24.328167,10.058504,3128.333333,6299.677849,6794.487251,2645.622044
2,602,8815,6437,-178,5835,-3635,2200,6013,178,5835,29.349835,12.211693,3068.333333,6237.204903,6742.308581,2619.198351
3,3448,8925,6389,-2912,2941,-3317,-376,5853,2912,2941,22.831406,10.459157,4035.000000,6897.756229,6732.685720,4239.671685
4,3332,8996,6453,-2848,3121,-3426,-305,5969,2848,3121,25.655101,11.503924,4028.666667,6937.191146,6806.816143,4159.846752
5,544,8767,6325,-238,5781,-3577,2204,6019,238,5781,40.314174,16.950359,3052.666667,6184.372644,6695.342336,2624.291523


# Stacking

In [159]:
# classifiers = []

# for name, p in pipelines.items():
#     if 'KNN'.lower() in name.lower():
#         continue
        
#     classifiers.append(p.steps[-1][1])

In [160]:
# %%time
# from mlxtend.classifier import EnsembleVoteClassifier

# eclf = EnsembleVoteClassifier(clfs=classifiers,
#                              voting='soft')


# score = fit_predict(eclf)

# print('score={}'.format(score))


In [161]:
# %%time
# from mlxtend.classifier import EnsembleVoteClassifier

# eclf = EnsembleVoteClassifier(clfs=classifiers,
#                              voting='hard')


# score = fit_predict(eclf)

# print('score={}'.format(score))
